In [12]:
## test for reading pre-processed rico dataset
## convert preprocessed data to seq format (with discreting process embedded)

import torch
import io
import random

with open('../raw_datasets/rico/pre_processed_20_25/test.pt', 'rb') as f:
    buffer = io.BytesIO(f.read())
    file=torch.load(buffer)

file[0]

{'name': '15468.json', 'bboxes': tensor([[0.3806, 0.3473, 0.2382, 0.0535],
        [0.2576, 0.1887, 0.0597, 0.0535],
        [0.3729, 0.1102, 0.2583, 0.0559],
        [0.0000, 0.8687, 0.2139, 0.0656],
        [0.2139, 0.8945, 0.5722, 0.0137],
        [0.7861, 0.8687, 0.2139, 0.0656],
        [0.1000, 0.0410, 0.8000, 0.5348],
        [0.0000, 0.5758, 1.0000, 0.0504],
        [0.0000, 0.6262, 1.0000, 0.0398]]), 'labels': tensor([ 2,  2,  2,  5, 16,  5,  2,  1,  1]), 'canvas_size': [1440.0, 2560.0], 'filtered': False}


In [14]:
import math
import copy
class AddGaussianNoise():
    '''
    Add Gaussian Noise to bounding box
    '''

    def __init__(self,
                 mean=0.,
                 std=1.,
                 normalized: bool = True,
                 bernoulli_beta: float = 1.0):
        self.std = std
        self.ori_std=std
        self.mean = mean
        self.normalized = normalized
        # adding noise to every element by default
        self.bernoulli_beta = bernoulli_beta
        print('Noise: mean={0}, std={1}, beta={2}'.format(
            self.mean, self.std, self.bernoulli_beta))
        if self.ori_std >= 1.0:
            print("mix noise!")
    def __call__(self, data):
        # Gold Label
        if 'gold_bboxes' not in data.keys():
            data['gold_bboxes'] = copy.deepcopy(data['bboxes'])

        num_elemnts = data['bboxes'].size(0)
        beta = data['bboxes'].new_ones(num_elemnts) * self.bernoulli_beta
        element_with_noise = torch.bernoulli(beta).unsqueeze(dim=-1)

        if self.ori_std>=3.0:
            rand=torch.rand(1).item()
            if rand<=0.2:
                self.std=0.005
            elif rand<=0.4 and rand>0.2:
                self.std=0.01
            elif rand<=0.6 and rand>0.4:
                self.std=0.015
            elif rand<=0.8 and rand>0.6:
                self.std=0.02
            else:
                self.std=0.025


        elif self.ori_std>=2.0:
            rand=torch.rand(1).item()
            if rand<=0.25:
                self.std=0.005
            elif rand<=0.50 and rand>0.25:
                self.std=0.01
            elif rand<=0.75 and rand>0.5:
                self.std=0.015
            else:
                self.std=0.02

        elif self.ori_std >= 1.0: #mix noise
            rand=torch.rand(1).item()
            if rand<=0.33:
                self.std=0.005
            elif rand<=0.66 and rand>0.33:
                self.std=0.01
            else:
                self.std=0.02


        if self.normalized:
            data['bboxes'] = data['bboxes'] + torch.randn(
                data['bboxes'].size()) * self.std + self.mean
        else:
            canvas_width, canvas_height = data['canvas_size'][0], data[
                'canvas_size'][1]
            ele_x, ele_y = data['bboxes'][:, 0] * canvas_width, data[
                'bboxes'][:, 1] * canvas_height
            ele_w, ele_h = data['bboxes'][:, 2] * canvas_width, data[
                'bboxes'][:, 3] * canvas_height
            data['bboxes'] = torch.stack([ele_x, ele_y, ele_w, ele_h], dim=1)
            data['bboxes'] = data['bboxes'] + torch.randn(
                data['bboxes'].size()) * self.std + self.mean
            data['bboxes'][:, 0] /= canvas_width
            data['bboxes'][:, 1] /= canvas_height
            data['bboxes'][:, 2] /= canvas_width
            data['bboxes'][:, 3] /= canvas_height
        data['bboxes'][data['bboxes'] < 0] = 0.0
        data['bboxes'][data['bboxes'] > 1] = 1.0
        data['bboxes'] = data['bboxes'] * element_with_noise + data[
            'gold_bboxes'] * (1 - element_with_noise)
        return data

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1}, beta={2})'.format(
            self.mean, self.std, self.bernoulli_beta)

def convert_ltwh_to_ltrb(bbox):
    l, t, w, h = decapulate(bbox)
    r = l + w
    b = t + h
    return torch.stack([l, t, r, b], axis=-1)
class LabelDictSort():
    '''
    sort elements in one layout by their label
    '''
    def __init__(self, index2label=None):
        self.index2label = index2label

    def __call__(self, data):
        # NOTE: for refinement
        if 'gold_bboxes' not in data.keys():
            data['gold_bboxes'] = copy.deepcopy(data['bboxes'])

        labels = data['labels'].tolist()
        idx2label = [[idx, self.index2label[labels[idx]]] for idx in range(len(labels))]
        idx2label_sorted = sorted(idx2label, key=lambda x : x[1])
        idx_sorted = [d[0] for d in idx2label_sorted]
        data['bboxes'], data['labels'] = data['bboxes'][idx_sorted], data['labels'][idx_sorted]
        data['gold_bboxes'] = data['gold_bboxes'][idx_sorted]
        return data

class LexicographicSort():
    '''
    sort elements in one layout by their top and left postion
    '''

    def __call__(self, data):
        if 'gold_bboxes' not in data.keys():
            data['gold_bboxes'] = copy.deepcopy(data['bboxes'])
        l, t, _, _ = data['bboxes'].t()
        _zip = zip(*sorted(enumerate(zip(t, l)), key=lambda c: c[1:]))
        idx = list(list(_zip)[0])
        data['ori_bboxes'], data['ori_labels'] = data['gold_bboxes'], data[
            'labels']
        data['bboxes'], data['labels'] = data['bboxes'][idx], data['labels'][
            idx]
        data['gold_bboxes'] = data['gold_bboxes'][idx]
        return data

def decapulate(bbox):
    if len(bbox.size()) == 2:
        x1, y1, x2, y2 = bbox.T
    else:
        x1, y1, x2, y2 = bbox.permute(2, 0, 1)
    return x1, y1, x2, y2

class DiscretizeBoundingBox():

    def __init__(self, num_x_grid: int, num_y_grid: int) -> None:
        self.num_x_grid = num_x_grid
        self.num_y_grid = num_y_grid
        self.max_x = self.num_x_grid - 1
        self.max_y = self.num_y_grid - 1

    def discretize(self, bbox):
        """
        Args:
            continuous_bbox torch.Tensor: N * 4
        Returns:
            discrete_bbox torch.LongTensor: N * 4
        """
        cliped_boxes = torch.clip(bbox, min=0.0, max=1.0)
        x1, y1, x2, y2 = decapulate(cliped_boxes)
        discrete_x1 = torch.floor(x1 * self.max_x)
        discrete_y1 = torch.floor(y1 * self.max_y)
        discrete_x2 = torch.floor(x2 * self.max_x)
        discrete_y2 = torch.floor(y2 * self.max_y)
        return torch.stack(
            [discrete_x1, discrete_y1, discrete_x2, discrete_y2],
            dim=-1).long()

    def discretize_num(self, num: float) -> int:
        return int(math.floor(num * self.max_y))

    def __call__(self, data):
        discrete_bboxes = self.discretize(data['bboxes'])
        data['discrete_bboxes'] = discrete_bboxes
        return data



In [15]:

labels = [
    'Text', 'Image', 'Icon', 'List_Item', 'Text_Button', 'Toolbar',
    'Web_View', 'Input', 'Card', 'Advertisement', 'Background_Image',
    'Drawer', 'Radio_Button', 'Checkbox', 'Multi_Tab', 'Pager_Indicator',
    'Modal', 'On_Off_Switch', 'Slider', 'Map_View', 'Button_Bar', 'Video',
    'Bottom_Navigation', 'Number_Stepper', 'Date_Picker'
]


In [16]:
idx2label={}
for i in range(25):
    idx2label[i+1]=labels[i]
sort=LexicographicSort()
sort_type=LabelDictSort(index2label=idx2label)
discrete_fn=DiscretizeBoundingBox(128,128)


In [17]:
gaussian=AddGaussianNoise(mean=0,std=0.005,normalized=True,bernoulli_beta=1.0)

for i in range(len(file)):

    ### sort ###
    file[i]=sort_type(file[i])
    # file[i]=sort(file[i])

    ### noise for refine ###
    # file[i]=gaussian(file[i])

    ### ltrb ###
    file[i]['bboxes']=convert_ltwh_to_ltrb(file[i]['bboxes'])

    ### discretize ###
    file[i]=discrete_fn(file[i])

print(file[0])

Noise: mean=0, std=0.005, beta=1.0
{'name': '15468.json', 'bboxes': tensor([[0.3806, 0.3473, 0.6188, 0.4008],
        [0.2576, 0.1887, 0.3174, 0.2422],
        [0.3729, 0.1102, 0.6313, 0.1660],
        [0.1000, 0.0410, 0.9000, 0.5758],
        [0.2139, 0.8945, 0.7861, 0.9082],
        [0.0000, 0.5758, 1.0000, 0.6262],
        [0.0000, 0.6262, 1.0000, 0.6660],
        [0.0000, 0.8687, 0.2139, 0.9344],
        [0.7861, 0.8687, 1.0000, 0.9344]]), 'labels': tensor([ 2,  2,  2,  2, 16,  1,  1,  5,  5]), 'canvas_size': [1440.0, 2560.0], 'filtered': False, 'gold_bboxes': tensor([[0.3806, 0.3473, 0.2382, 0.0535],
        [0.2576, 0.1887, 0.0597, 0.0535],
        [0.3729, 0.1102, 0.2583, 0.0559],
        [0.1000, 0.0410, 0.8000, 0.5348],
        [0.2139, 0.8945, 0.5722, 0.0137],
        [0.0000, 0.5758, 1.0000, 0.0504],
        [0.0000, 0.6262, 1.0000, 0.0398],
        [0.0000, 0.8687, 0.2139, 0.0656],
        [0.7861, 0.8687, 0.2139, 0.0656]]), 'discrete_bboxes': tensor([[ 48,  44,  78,  50],


In [19]:
all_layout=[]

for i,_ in enumerate(file):
    
    sample=file[i]
    layout=[]

   # layout.append('START')

    bbox=sample['discrete_bboxes']
    for idx,label in enumerate(sample['labels']):
        layout.append(labels[label-1])
        for pos in bbox[idx]:
            layout.append(str(pos.numpy()))
        if idx!=len(bbox)-1:
            layout.append("|")
    
    # layout.append('END ]')

    str_layout=' '.join(layout)
    all_layout.append(str_layout)


In [20]:
# # for edit
# all_layout=[]
# random.seed(10)
# for i,_ in enumerate(file):
    
#     sample=file[i]
#     layout=[]

#     len_layout=len(sample['labels'])
#     layout.append('START')

#     bbox=sample['discrete_bboxes']
#     for idx,label in enumerate(sample['labels']):
#         if len_layout>1 and random.random()>0.8: # omit 80% elements
#             if idx==len(bbox)-1 and len(layout)!=0:
#                 if layout[-1]=="|":
#                     layout.pop()
#             continue
#         layout.append(labels[label-1])
#         for pos in bbox[idx]:
#             layout.append(str(pos.numpy()))
#         if idx!=len(bbox)-1:
#             layout.append("|")
        
    
#     layout.append('END ]')

#     str_layout=' '.join(layout)
#     all_layout.append(str_layout)

In [21]:
print(all_layout[0])
print(all_layout[1])

START Image 48 44 78 50 | Image 32 23 40 30 | Image 47 13 80 21 | Image 12 5 114 73 | Text 0 79 127 84 | Text_Button 0 110 27 118 | Text_Button 99 110 127 118 END ]
START Image 0 4 127 24 | Input 3 35 123 44 | Text_Button 3 70 123 79 END ]


In [22]:
data=open("../data/processed_datasets/RICO_ltrb_lex_edit/src1_test_draw.txt",'w+')
for _,layout in enumerate(all_layout): 
    print(layout,file=data)
data.close()